In [1]:
# !pip install transformers -U

In [2]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from sentence_transformers import SentenceTransformer
# util
from math import sqrt, pow, exp
import numpy as np

In [3]:

DATA_COLUMN = 1
LABEL_COLUMN = 0
df_ORI = pd.read_csv("data/data_training_90.txt", sep="\t", header=None)#, encoding="UTF-16")
df_ORI.columns = [LABEL_COLUMN, DATA_COLUMN]
df_AUG = pd.read_csv("data/data_training_90_1_a_NO_RS(augonly).txt", sep="\t", header=None)
df_AUG.columns = [LABEL_COLUMN, DATA_COLUMN]

In [4]:
cos_model_name = "distiluse-base-multilingual-cased-v2"

In [5]:
def jaccard_similarity(x,y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)
def squared_sum(x):
    """ return 3 rounded square rooted value """
    
    return round(sqrt(sum([a*a for a in x])),3)
 
def euclidean_distance(x,y):
    """ return euclidean distance between two lists """
    
    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

def distance_to_similarity(distance):
    return 1/exp(distance)

def cos_similarity(x,y):
    """ return cosine similarity between two lists """    
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = squared_sum(x)*squared_sum(y)
    return round(numerator/float(denominator),3)

def create_embeddings (text, SentenceTransformer_model): 
    embeddings = SentenceTransformer_model.encode(text)
    if len(embeddings) !=0:
        return list(embeddings)
    else:
        return [0]
def calculate_bleu_scores(references, hypotheses):
    """
    Calculates BLEU 1-4 scores based on NLTK functionality

    Args:
        references: List of reference sentences
        hypotheses: List of generated sentences

    Returns:
        bleu_1, bleu_2, bleu_3, bleu_4: BLEU scores

    """
    #return len(references), len(hypotheses)
    bleu_1 = np.round(corpus_bleu(references, hypotheses, weights=(1.0, 0., 0., 0.)), decimals=2)
    bleu_2 = np.round(corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0., 0.)), decimals=2)
    bleu_3 = np.round(corpus_bleu(references, hypotheses, weights=(0.34, 0.33, 0.33, 0.)), decimals=2)
    bleu_4 = np.round(corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25)), decimals=2)
    return bleu_1, bleu_2, bleu_3, bleu_4 

In [6]:
SentenceTransformer_model = SentenceTransformer(cos_model_name)
df           = pd.DataFrame()
newDF        = pd.DataFrame()
df           = df_ORI
#df.reset_index(inplace=True)
cntr         = 1

for index, row in df.iterrows():
    print('Rows Counter: ' + str(cntr))
    text     = row[DATA_COLUMN]
    all_text = df_AUG.iloc[index][DATA_COLUMN]
    embd1 = create_embeddings(text=text, SentenceTransformer_model=SentenceTransformer_model)
    embd2 = create_embeddings(text=all_text, SentenceTransformer_model=SentenceTransformer_model)
    new_embd1 = ','.join(str(x) for x in embd1)
    new_embd2 = ','.join(str(x) for x in embd2)
    esim = euclidean_distance(embd1, embd2)
    csim = cos_similarity(embd1, embd2)
    jsim = jaccard_similarity(text, all_text)    
    tmp = { 
        'text': [row[DATA_COLUMN]], 
        'label': [row[LABEL_COLUMN]], 
        'all_text': [all_text], 
        'original_embedding': [new_embd1], 
        'new_embedding': [new_embd2], 
        'ecu_similarity': [esim], 
        'cos_similarity': [csim], 
        'jacc_similarity': [jsim] 
    }
    tmpDF = pd.DataFrame(tmp)
#     print(tmpDF)
    newDF = pd.concat([newDF, tmpDF], ignore_index=True)
    
    cntr = cntr + 1
    #if cntr == 10:
#     #    break;
        
#     # GPU Monitor
#     if index%10 == 0:
#         GPUs = GPU.getGPUs()  
#         gpu = GPUs[0]    
#         print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))        
#         torch.cuda.empty_cache()


model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

C:\Users\Server\miniconda3\envs\RALBERT\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Server\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

Rows Counter: 1
Rows Counter: 2
Rows Counter: 3
Rows Counter: 4
Rows Counter: 5
Rows Counter: 6
Rows Counter: 7
Rows Counter: 8
Rows Counter: 9
Rows Counter: 10
Rows Counter: 11
Rows Counter: 12
Rows Counter: 13
Rows Counter: 14
Rows Counter: 15
Rows Counter: 16
Rows Counter: 17
Rows Counter: 18
Rows Counter: 19
Rows Counter: 20
Rows Counter: 21
Rows Counter: 22
Rows Counter: 23
Rows Counter: 24
Rows Counter: 25
Rows Counter: 26
Rows Counter: 27
Rows Counter: 28
Rows Counter: 29
Rows Counter: 30
Rows Counter: 31
Rows Counter: 32
Rows Counter: 33
Rows Counter: 34
Rows Counter: 35
Rows Counter: 36
Rows Counter: 37
Rows Counter: 38
Rows Counter: 39
Rows Counter: 40
Rows Counter: 41
Rows Counter: 42
Rows Counter: 43
Rows Counter: 44
Rows Counter: 45
Rows Counter: 46
Rows Counter: 47
Rows Counter: 48
Rows Counter: 49
Rows Counter: 50
Rows Counter: 51
Rows Counter: 52
Rows Counter: 53
Rows Counter: 54
Rows Counter: 55
Rows Counter: 56
Rows Counter: 57
Rows Counter: 58
Rows Counter: 59
Rows C

In [7]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
embeddings = model.encode(sentences)
print(embeddings)

[[-0.03885619  0.01854845 -0.04066142 ...  0.01009195 -0.01660531
  -0.00138946]
 [-0.00059496 -0.009242   -0.0587051  ...  0.01638779  0.01509569
  -0.04368324]]


In [8]:
newDF['text_split'] = [list(x.split()) for x in newDF['text']]
newDF['all_text_split'] = [x.split() for x in newDF['all_text']]
# df['new_text_split'] = [str(x).split() for x in df['new_text']]

In [9]:
newDF[['bleu_sim_1','bleu_sim_2','bleu_sim_3','bleu_sim_4']] = [ calculate_bleu_scores ([[x]],[y]) for x, y in zip(newDF['text_split'], newDF['all_text_split'])]

C:\Users\Server\miniconda3\envs\RALBERT\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Server\miniconda3\envs\RALBERT\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Server\miniconda3\envs\RALBERT\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-g

In [10]:
newDF.head()

,text,label,all_text,original_embedding,new_embedding,ecu_similarity,cos_similarity,jacc_similarity,text_split,all_text_split,bleu_sim_1,bleu_sim_2,bleu_sim_3,bleu_sim_4
0,lucu kak abbey cantik kayak orang haru,1,lucu kak abbey cantik kayak adiwarna orang haru,"-0.008952065,0.04007555,0.009347842,-0.0112527...","-0.010526381,0.03891554,0.0057675247,-0.013351...",0.071316,0.991,0.888889,"[lucu, kak, abbey, cantik, kayak, orang, haru]","[lucu, kak, abbey, cantik, kayak, adiwarna, or...",0.88,0.79,0.68,0.59
1,heran deh ngatain baperan wajar dese baper udi...,0,heran deh ngatain baperan wajar dese baper udi...,"-0.0025755763,0.02613189,0.007338449,-0.015779...","-0.005544476,0.02702064,0.0063123675,-0.014782...",0.040416,0.997,1.000000,"[heran, deh, ngatain, baperan, wajar, dese, ba...","[heran, deh, ngatain, baperan, wajar, dese, ba...",1.00,0.94,0.90,0.86
2,jujur ni org bagus kaga oplasada tu foto sblm ...,0,jujur ni org bagus kaga oplasada tu foto sblm ...,"-0.03544854,-0.0025185551,0.0033979164,-0.0242...","-0.03311891,-0.0023508081,-0.006632578,-0.0260...",0.124963,0.970,0.954545,"[jujur, ni, org, bagus, kaga, oplasada, tu, fo...","[jujur, ni, org, bagus, kaga, oplasada, tu, fo...",0.94,0.90,0.86,0.84
3,haduuh orang gak akhlak bangeg kerja antem mul...,0,haduuh orang gak budi bahasa bangeg kerja ante...,"0.00045949852,0.0074383696,0.011340753,-0.0033...","0.009412642,0.018586768,0.012307178,0.00205228...",0.270810,0.822,0.944444,"[haduuh, orang, gak, akhlak, bangeg, kerja, an...","[haduuh, orang, gak, budi, bahasa, bangeg, ker...",0.80,0.73,0.65,0.53
4,bingung jujur media rebut ngeliput orang kayak...,0,bingung jujur media jaring rebut ngeliput oran...,"-0.030819902,0.036678318,-0.022147873,-0.02612...","-0.03201777,0.03360407,-0.026444586,-0.0279051...",0.056634,0.996,1.000000,"[bingung, jujur, media, rebut, ngeliput, orang...","[bingung, jujur, media, jaring, rebut, ngelipu...",0.92,0.88,0.82,0.76


In [11]:
newDF.to_csv("data/testing_distiluse-base-multilingual-cased.txt", sep="\t", header=0,encoding='utf-16', index=False)

In [12]:
SIM_COFFICIENTS_THRESHOLDS = {'ECU': newDF["ecu_similarity"].mean(), 'COS':newDF["cos_similarity"].mean(), 'JAC':newDF["jacc_similarity"].mean(), 'BLEU':newDF["bleu_sim_1"].mean()}
SIM_COFFICIENTS_THRESHOLDS

{'ECU': 0.15499225413325277,
 'COS': 0.9529583333333334,
 'JAC': 0.953690701511386,
 'BLEU': 0.9102500000000001}